
# Arguments

In [ ]:
num_of_epochs = 100
batch_size = 32
dataset_name = 'plantVillage-tomato-mohanty'
dataset_path = '../datasets/' + dataset_name
model_save_path = 'vgg-model.h5'


model_diagram_path = 'vgg_model_diagram.png'
plot_save_path = 'plot-plantVillage-alexnet-from-scratch.png'

input_width = 224
input_height = 224
input_depth = 3



# Load model

In [ ]:
from keras.applications import VGG16

# Load pretrianed VGG model with FC layers removed
# explicitly deﬁne the input_tensor to be 224×224×3 pixels
baseModel = VGG16(weights='imagenet',include_top=False,
                  input_tensor=Input(shape = (input_width,input_height, input_depth)))

# Custom FC layer

In [ ]:
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense

class FCHeadNet:
    @staticmethod
    def build(baseModel,classes,D):
        # Initialize the headModel and build this simple architecture
        # INPUT => FC => RELU => DO => FC => SOFTMAX
        
        headModel = baseModel.output
        headModel = Flatten(name='flatten')(headModel)
        headModel = Dense(D,activation='relu')(headModel)
        headModel = Dropout(0.5)(headModel)
        # Add a softmaxc layer
        headModel = Dense(classes,activation='softmax')(headModel)
        return headModel

# prepare dataset for trainining

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator # 
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

print("[INFO] loading images...")
imagePaths = list(paths.list_images(args['dataset']))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]


# Initial image preprocessing
aap = AspectAwarePreprocesser(224,224)
iap= ImageToArrayPreprocess()

#Load image data and perform image data preprocessing
sdl = SimpleDatasetLoader(preprocessors=[aap,iap])
(data,labels)  = sdl.load(imagePaths,verbose=500)
data = data.astype("float") / 255.0

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX,testX,trainY,testY) = train_test_split(data,labels,
	test_size=0.25,random_state=42)

# convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Attach custom head to model

In [ ]:
# Initialize a new fully connected layer
headModel = FCHeadNet.build(baseModel,len(classNames),256)

# place the head FC model on top of the base model 
model = Model(inputs=baseModel.input,outputs = headModel)

# traverse all layers and freeze the weight of the corresponding layer
for layer in baseModel.layers:
    layer.trainable = False

# Warm up head

In [ ]:
print("[INFO] compiling model...")
opt = RMSprop(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
# Since we only train the new fully connected layer, we do a few iterations
print("[INFO] training head...")
model.fit_generator(aug.flow(trainX,trainY,batch_size = 32),
                             validation_data = (testX,testY),epochs=25,
                             steps_per_epoch = len(trainX) //32,verbose = 1)

# Evaluate after warmup

In [ ]:
# 
print("[INFO] evaluating after initialization...")
predictions = model.predict(testX,batch_size=32)
print(classification_report(testY.argmax(axis =1),
                            predictions.argmax(axis =1),target_names=classNames))

# Unfreeze some conv layers

In [ ]:
for layer in baseModel.layers[15:]:
  layer.trainable = True

# Train model again

In [ ]:
# This time we trainfor 100 epochs

# Build the model from the new
print("[INFO] re-compiling model ...")

from keras.optimizers import SGD
opt = SGD(lr=0.001)
# Fine-tuning with a small learning rate
model.compile(loss = 'categorical_crossentropy',optimizer = opt,
              metrics=['accuracy'])
# fine-tuning the entire model
print("[INFO] fine-tuning model...")
model.fit_generator(aug.flow(trainX,trainY,batch_size=32),
                    validation_data = (testX,testY),epochs = 100,
                    steps_per_epoch = len(trainX) // 32,verbose = 1)

# Evaluate trained model

In [ ]:
# Evaluation of fine-tuned model results
print("[INFO] evaluating after fine-tuning...")
predictions = model.predict(testX,batch_size=32)
print(classification_report(testY.argmax(axis =1),
        predictions.argmax(axis =1),target_names=classNames))

# Save the model to disk
print("[INFO] serializing model...")
model.save(args['model'])